## Imports

In [23]:
from ultralytics import YOLO
import cv2 
from matplotlib import pyplot as plt
import time
import psutil
import sys
import threading
from queue import Queue

## Load model 

In [24]:
# Load models
v8n_person = YOLO("train_runs/yolov8n_person_100epochs/detect/train/weights/best.pt")
v8n_mask = YOLO("train_runs/yolov8n_mask_100epochs/train/weights/best.pt")
v11n_person = YOLO("train_runs/yolo11n_person_100epochs/train/weights/best.pt")
v11n_mask = YOLO("train_runs/yolo11n_mask_100epochs/train/weights/best.pt")

# Create model list
model_dict = {"v8n_person":v8n_person, "v8n_mask":v8n_mask, "v11n_person":v11n_person, "v11n_mask":v11n_mask}

## Test if model was loaded correctly

In [25]:
# Function to plot result images
def matplot_image(results: list, model_name: str):
    """
        Display the image using matplotlib
        :param results: list of results from the model
        :param model_name: name of the model
    """

    im  = results[0].plot()
    # Convert BGR (OpenCV format) to RGB (matplotlib format)
    im_rgb = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)

    # Add model name to the image
    plt.title(model_name)

    # Display the image using matplotlib
    plt.imshow(im_rgb)
    plt.axis("off")  # Turn off axes for a cleaner display
    plt.show()


In [26]:
# Set confidence and IOU threshold
conf = 0.3
iou = 0.3

# Test models
for model_name, model in model_dict.items():
    for i in range(3):
        results = model.predict(f"data_sets/image_data/mask_person_test/{i}.jpg", conf=conf, iou=iou, verbose=False)

        # Show results
        #matplot_image(results, model_name)

## Get RAM usage

In [27]:
# Get RAM usage
def get_ram_usage():
    """
        Get the RAM usage of the system
        :return: RAM usage in MB
    """


    # Get current process
    process = psutil.Process()

    # Get memory usage in bytes and convert to MB
    return process.memory_info().rss / (1024 ** 2)  # Resident Set Size (RSS)
    

## Put stats bar on frame

In [28]:
# Put stats bar on frame
def put_stats_bar(frame, t0_inf=0, t1_inf=0, t0_ann=0, t1_ann=0, loop_time=0, bar_height=30,font_scale=0.5, font_thickness=1):
    """
        Add a stats bar at the bottom of the frame
        :param frame: frame to add the stats bar to
        :param t0_inf: start time of inference
        :param t1_inf: end time of inference
        :param t0_ann: start time of annotation
        :param t1_ann: end time of annotation
        :param loop_time: time taken to process the frame
        :param bar_height: height of the stats bar
        :param font_scale: font scale of the text
        :param font_thickness: font thickness of the text
    """

    # Calculate inference time
    inference_time = (t1_inf - t0_inf) * 1000  # Convert to milliseconds
    
    # Calculate annotation time
    annotation_time = (t1_ann - t0_ann) * 1000  # Convert to milliseconds

    # Calculate max FPS
    max_fps = 1 / loop_time if loop_time > 0 else 0

    # Add text inside the rectangle
    stats_text = (
    f"Inference time: {inference_time:.2f} ms | "
    f"Annotation time: {annotation_time:.2f} ms | "
    f"Max FPS: {max_fps:.2f}"
    f"      Press 'q' to quit | Press '+' to increase confidence | Press '-' to decrease confidence"
    )
        
    # Add a rectangle at the bottom of the frame
    h, w, _ = frame.shape  # Get frame dimensions
    cv2.rectangle(frame, (0, h - bar_height), (w, h), (0, 0, 0), -1)  # Black rectangle

    # Add text inside the rectangle
    text_color = (255, 255, 255)  # White text
    cv2.putText(frame, stats_text, (10, h - 10), cv2.FONT_HERSHEY_SIMPLEX, font_scale, text_color, font_thickness)

## Test Webcam as input

In [ ]:
# Set confidence and IOU threshold
conf = 0.3
iou = 0.3
model = model_dict["v8n_mask"]

# Inference on video
results = model.predict(source=0, stream=True, conf=conf, iou=iou, verbose=False)

# Create frame counter
frame_count = 0

for result in results:
    # Extract the processed frame with predictions
    t0 = time.time()    # Start time
    frame = result.plot()  # Visualizes predictions on the frame
    t1 = time.time()    # End time

    # Calculate inference 
    inference_time = (t1 - t0) * 1000  # Convert to milliseconds

    # Calculate max FPS
    max_fps = 1000 / inference_time

    # Get RAM usage
    ram_usage = get_ram_usage()

    # Add text inside the rectangle
    if frame_count % 20 == 0:
        stats_text = (
        f"Inference time: {inference_time:.2f} ms | "
        f"RAM usage: {ram_usage:.2f} MB | "
        f"Max FPS: {max_fps:.2f}"
    )
        # Reset frame count
        frame_count = 0

    # Add stats bar to the frame
    put_stats_bar(frame=frame, bar_height=30, font_scale=0.5, font_thickness=1)
    
    # Display the frame in a window
    cv2.imshow('Live Predictions', frame)
    
    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    frame_count += 1
    

cv2.destroyAllWindows()

1/1: 0... Success ✅ (inf frames of shape 1920x1080 at 24.00 FPS)

